<a href="https://colab.research.google.com/github/David855008/BA865/blob/main/BA870_Assignment_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

###Ta-Wei Wang 
####davidtww@bu.edu
#### BU ID: U88201119

#This is a sample template to help you start working on Assignment 1

### You should first read the instructions for Assignment 1 on the BA870 site on Blackboard (under the Assignments tab).
### Also, look at the lecture notes and Echo360 video for Lecture 2 to see relevant comments on scraping the SEC website.


* This short Colab Notebook provides some example Python code to download the financial statement numbers for Apple for its most recent Income Statements (Consolidated Statement of Operations).
* The SEC URL for this information for Apple is:
https://www.sec.gov/Archives/edgar/data/320193/000032019321000105/R2.htm



* This example demonstrates how to find the key identifying "tag" to isolate the key text from a webpage.

In [315]:
# Install necessary Python libraries
! pip install requests
! pip install beautifulsoup4
! pip install bs4

##Use the "requests" library to download the necessary webpage from thge SEC server. 

In [316]:
import requests
import numpy as np
import pandas as pd
import string
import re
import time

In [317]:
# This header information MUST be submitted with your URL reuqest to the SEC website.
# If this header is not included or is incorrect, then you will not be abkle to download the data (it will be rejected by the SEC server).

heads = {'Host': 'www.sec.gov', 'Connection': 'close',
         'Accept': 'application/json, text/javascript, */*; q=0.01', 'X-Requested-With': 'XMLHttpRequest',
         'User-Agent': 'davidw70901@gmail.com',
         }

URL = 'https://www.sec.gov/Archives/edgar/data/320193/000032019321000105/R2.htm'

# Get data from webpage
page = requests.get(URL, headers=heads)
time.sleep(1)

## Use "Beautiful Soup" Library to implement an "html/xml parser"

Here is the link to the documentation:
https://www.crummy.com/software/BeautifulSoup/bs4/doc/

In this example we will download the Income Statement for Apple Inc. (AAPL) from the sec website.


In [318]:
from bs4 import BeautifulSoup
soup = BeautifulSoup(page.content, 'lxml')

In [319]:
# The Prettify function formats that html/xml text to make it easier to read.
# You would inspect the html code to find marker tags that identifies the specific section of text you wish to extract.
# print(soup.prettify())

## The Next Step is to Find the HTML Tags
### The tag  ***\<td class="nump"*** precedes any reported numbers AAPL's financial statements 

In [320]:
# soup.find_all(class_='nump')

## Print out the scraped Numbers that had the class tag "nump" from  Apple's Income Statement 

In [321]:
# Find the tag that identify numbers from Apple's Income Statement 
Numbers = soup.find_all ('td', class_='nump')

# To Do Part:

## Defining the columns:

In [322]:
# For dates, find all <th> tags and remove the unwanted html codes
Dates = soup.find_all ('th', class_='th')[1:]
dates = [s.__str__().replace('<th class="th"><div>','').replace('</div></th>','').replace('.','').replace(',','') for s in Dates]

In [323]:
# column_names contains the columns for the final dataframe to create 
column_names = ['Income Statement Item for Apple'] + dates

## Create the DataFrame

In [324]:
# create the DataFrame
ans = pd.DataFrame(columns = column_names)
# find the page data with tr tag (it contains the income statement details)
page_data = soup.find_all(class_='report')[0].find_all('tr')
# the first 2 rows are the title and the date, so starting from the third row
for i in page_data[2:len(page_data)]:
  temp = i.find_all('td')
  # create a list of the Items and numbers accordingly to append 
  newrow = [temp[0].text]
  for j in temp[1:]:
    # using regular expression to distinguish the floating numbers
    input = re.sub('[^0-9.]+', '',j.text)
    # if the element is empty, append nan, else append the floating type element
    if input == '':newrow.append(float("nan"))
    else:newrow.append(float(input))
  # append the row at the bottom of the DataFrame
  ans.loc[len(ans)] = newrow

## Results

In [325]:
# print the results
ans

,Income Statement Item for Apple,Sep 25 2021,Sep 26 2020,Sep 28 2019
0,Net sales,365817.00,274515.00,260174.00
1,Cost of sales,212981.00,169559.00,161782.00
2,Gross margin,152836.00,104956.00,98392.00
3,Operating expenses:,NaN,NaN,NaN
4,Research and development,21914.00,18752.00,16217.00
5,"Selling, general and administrative",21973.00,19916.00,18245.00
6,Total operating expenses,43887.00,38668.00,34462.00
7,Operating income,108949.00,66288.00,63930.00
8,"Other income/(expense), net",258.00,803.00,1807.00
9,Income before provision for income taxes,109207.00,67091.00,65737.00


# Microsoft(MSFT) for validation

In [326]:
# This header information MUST be submitted with your URL reuqest to the SEC website.
# If this header is not included or is incorrect, then you will not be abkle to download the data (it will be rejected by the SEC server).

heads = {'Host': 'www.sec.gov', 'Connection': 'close',
         'Accept': 'application/json, text/javascript, */*; q=0.01', 'X-Requested-With': 'XMLHttpRequest',
         'User-Agent': 'davidw70901@gmail.com',
         }

URL = 'https://www.sec.gov/Archives/edgar/data/789019/000156459021039151/R2.htm'

# Get data from webpage
page = requests.get(URL, headers=heads)
time.sleep(1)

In [327]:
soup = BeautifulSoup(page.content, 'lxml')

In [328]:
# For dates, find all <th> tags and remove the unwanted html codes
Dates = soup.find_all ('th', class_='th')[1:]
dates = [s.__str__().replace('<th class="th"><div>','').replace('</div></th>','').replace('.','').replace(',','') for s in Dates]
# column_names contains the columns for the final dataframe to create 
column_names = ['Income Statement Item for MSFT'] + dates

In [329]:
# create the DataFrame
ans2 = pd.DataFrame(columns = column_names)
# find the page data with tr tag (it contains the income statement details)
page_data = soup.find_all(class_='report')[0].find_all('tr')
# the first 2 rows are the title and the date, so starting from the third row
for i in page_data[2:len(page_data)]:
  temp = i.find_all('td')
  # create a list of the Items and numbers accordingly to append 
  newrow = [temp[0].text]
  for j in temp[1:]:
    # using regular expression to distinguish the floating numbers
    input = re.sub('[^0-9.]+', '',j.text)
    # if the element is empty, append nan, else append the floating type element
    if input == '':newrow.append(float("nan"))
    else:newrow.append(float(input))
  # append the row at the bottom of the DataFrame
  ans2.loc[len(ans2)] = newrow

In [330]:
ans2

,Income Statement Item for MSFT,Jun 30 2021,Jun 30 2020,Jun 30 2019
0,Revenue,168088.00,143015.00,125843.00
1,Cost of revenue,52232.00,46078.00,42910.00
2,Gross margin,115856.00,96937.00,82933.00
3,Research and development,20716.00,19269.00,16876.00
4,Sales and marketing,20117.00,19598.00,18213.00
5,General and administrative,5107.00,5111.00,4885.00
6,Operating income,69916.00,52959.00,42959.00
7,"Other income, net",1186.00,77.00,729.00
8,Income before income taxes,71102.00,53036.00,43688.00
9,Provision for income taxes,9831.00,8755.00,4448.00
